In [67]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time
import dask.dataframe as dd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
#Читаем тренировочные данные
data_train = pd.read_csv('data_train.csv')
# data_train["buy_time_format"] = data_train["buy_time"].map(lambda b: datetime.fromtimestamp(b))
data_train.pop('Unnamed: 0')
data_train.head()

,id,vas_id,buy_time,target
0,540968,8.0,1537131600,0.0
1,1454121,4.0,1531688400,0.0
2,2458816,1.0,1534107600,0.0
3,3535012,5.0,1535922000,0.0
4,1693214,1.0,1535922000,0.0


In [3]:
#Количество уникальных клиентов в тренировочных данных
len(data_train['id'].unique())

806613

In [4]:
# data_types ={'id': str,'buy_time': str}
# for i in range(253):
# #     print(f"col{i}")
#     data_types[f"col{i}"] = str
# dtype = data_types,
# features = dd.read_csv("G:/temp/features.csv/features.csv", delimiter="\t", blocksize=640000000)
#.compute()
# features.tail()

# len(features['id'].unique())
# 4362739
# len(features.index)
# 4512528

# список id из большого csv
# lst2 = list(features['id'])
# len(lst2)
# список id из тренировочного csv
# lst = list(data_train['id'])
# len(lst)

In [5]:
# Будем читать по частям
chunksize = 10 ** 5
filename = "G:/temp/features.csv/features.csv"
list_of_dfs = []
for chunk in pd.read_csv(filename, chunksize=chunksize, delimiter="\t"):
    chunk_id = chunk[['id','buy_time']]
    chunk_id['index_features'] = chunk_id.index
    list_of_dfs.append(chunk_id)

features_index = pd.concat(list_of_dfs)
# chunk.tail()
features_index.tail()

<ipython-input-5-adb84ea917a8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_id['index_features'] = chunk_id.index


,id,buy_time,index_features
4512523,3521417,1548018000,4512523
4512524,3521563,1547413200,4512524
4512525,3521574,1546808400,4512525
4512526,3521640,1547413200,4512526
4512527,3521702,1548018000,4512527


In [6]:
# Надо соединить по ближайшему значению поля buy_time
# можно соединить по id. потом посчитать разницу между buy_time и взять строки с минимальной разницей
features_train = pd.merge(left=data_train, left_on=['id'], right=features_index, right_on=['id'], how='inner')
features_train.tail()

,id,vas_id,buy_time_x,target,buy_time_y,index_features
860047,3812226,2.0,1546203600,0.0,1532898000,3598646
860048,2480469,2.0,1546203600,0.0,1532293200,1126545
860049,158236,2.0,1546203600,0.0,1538946000,3755060
860050,1825525,2.0,1546203600,0.0,1533502800,253884
860051,3022610,2.0,1546203600,0.0,1540760400,2556316


In [7]:
features_train['time_diff'] = abs(features_train['buy_time_x'] - features_train['buy_time_y'])

In [8]:
#Группируем по id, vas_id агрегатная функция: time_diff --> min
ftgb = features_train.groupby(['id', 'vas_id']).time_diff.agg(time_diff  = 'min')
ftgb

,,time_diff
id,vas_id,
2,2.0,0
4,1.0,4838400
15,1.0,2419200
16,2.0,7257600
29,1.0,604800
...,...,...
4362634,1.0,1209600
4362640,2.0,0
4362647,6.0,8467200


In [9]:
# у нас есть ключ для нужных нам строк: id, vas_id, min_time_diff
# Фильтруем features_train оставляя только нужные строки

In [10]:
features_train_2 = pd.merge(left=ftgb, left_on=['id', 'vas_id', 'time_diff'], right=features_train, right_on=['id', 'vas_id', 'time_diff'], how='inner')
features_train_2

,id,vas_id,time_diff,buy_time_x,target,buy_time_y,index_features
0,2,2.0,0,1545598800,0.0,1545598800,2966746
1,4,1.0,4838400,1533502800,0.0,1538341200,1005270
2,15,1.0,2419200,1534107600,0.0,1531688400,3686924
3,16,2.0,7257600,1540760400,0.0,1548018000,1747371
4,29,1.0,604800,1533502800,0.0,1532898000,1321339
...,...,...,...,...,...,...,...
825873,4362634,1.0,1209600,1546203600,0.0,1544994000,3822180
825874,4362640,2.0,0,1546203600,0.0,1546203600,689078
825875,4362647,6.0,8467200,1546203600,0.0,1537736400,2063477
825876,4362684,5.0,6652800,1545598800,0.0,1538946000,3035193


In [11]:
#для обучения нам не нужны колонки time_diff buy_time_y
features_train_2.drop(['time_diff', 'buy_time_y'], axis = 1)

,id,vas_id,buy_time_x,target,index_features
0,2,2.0,1545598800,0.0,2966746
1,4,1.0,1533502800,0.0,1005270
2,15,1.0,1534107600,0.0,3686924
3,16,2.0,1540760400,0.0,1747371
4,29,1.0,1533502800,0.0,1321339
...,...,...,...,...,...
825873,4362634,1.0,1546203600,0.0,3822180
825874,4362640,2.0,1546203600,0.0,689078
825875,4362647,6.0,1546203600,0.0,2063477
825876,4362684,5.0,1545598800,0.0,3035193


In [12]:
#нужно соединиться с дата сетом из файла
chunksize = 10 ** 5
filename = "G:/temp/features.csv/features.csv"
list_of_dfs = []
for chunk in pd.read_csv(filename, chunksize=chunksize, delimiter="\t"):
    chunk_merge = pd.merge(left=features_train_2, left_on=['index_features'], right=chunk, right_index=True, how='inner')
    list_of_dfs.append(chunk_merge)

features_train = pd.concat(list_of_dfs)
# chunk.tail()
features_train.tail()

,id_x,vas_id,time_diff,buy_time_x,target,buy_time_y,index_features,Unnamed: 0,id_y,buy_time,...,243,244,245,246,247,248,249,250,251,252
668999,3513825,1.0,13305600,1533502800,0.0,1546808400,4512493,4512493,3513825,1546808400,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
669006,3513869,1.0,15724800,1532293200,0.0,1548018000,4512494,4512494,3513869,1548018000,...,-928.373846,-570.770792,-23.996269,-37.630448,-271.747724,-22.832889,-0.694428,-12.175933,-0.45614,0.0
669611,3517434,2.0,2419200,1545598800,0.0,1548018000,4512507,4512507,3517434,1548018000,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
670020,3519714,2.0,1209600,1545598800,0.0,1546808400,4512515,4512515,3519714,1546808400,...,-977.373846,-613.770792,-14.996269,7.369552,-180.747724,-19.832889,0.305572,-12.175933,-0.45614,0.0
670310,3521152,1.0,14515200,1533502800,0.0,1548018000,4512521,4512521,3521152,1548018000,...,3312.626154,2447.229208,-22.996269,339.369552,17256.252276,-0.832889,0.305572,-12.175933,-0.45614,0.0


In [13]:
#Удалим лишние колонки
features_train = features_train.drop(['id_x', 'time_diff', 'buy_time_x', 'buy_time_y', 'index_features', 'Unnamed: 0', 'id_y'], axis = 1)
features_train[['target']] = features_train[['target']].astype(int)
features_train

,vas_id,target,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
8,1.0,0,1545598800,-96.799971,-408.179112,-110.740786,-460.786798,-116.158246,-481.89179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
51,1.0,0,1538946000,-45.949971,-357.329112,-48.900786,-398.946798,-54.318246,-420.05179,-16.08618,...,1027.626154,887.229208,-25.996269,-34.630448,-204.747724,-19.832889,-0.694428,-6.175933,-0.45614,0.0
66,1.0,0,1534712400,182.860029,365.560888,171.459214,376.513202,166.041754,397.78821,-16.08618,...,-975.373846,-613.770792,-24.996269,-2.630448,-139.747724,24.167111,-0.694428,35.824067,1.54386,0.0
144,1.0,0,1539550800,276.080029,-35.299112,262.139214,-87.906798,256.721754,-109.01179,-16.08618,...,-553.373846,-536.770792,-25.996269,-37.630448,-272.747724,-21.832889,-0.694428,-10.175933,-0.45614,0.0
156,1.0,0,1546808400,-96.799971,-364.429112,-110.740786,-412.796798,-41.158246,-182.22179,-16.08618,...,-278.373846,-613.770792,-25.996269,-12.630448,-256.747724,-23.832889,-0.694428,-12.175933,-0.45614,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668999,1.0,0,1546808400,-9.299971,-231.429112,26.779214,-183.566798,21.361754,-204.67179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
669006,1.0,0,1548018000,82.370029,-123.429112,155.939214,-88.526798,150.521754,-109.63179,-16.08618,...,-928.373846,-570.770792,-23.996269,-37.630448,-271.747724,-22.832889,-0.694428,-12.175933,-0.45614,0.0
669611,2.0,0,1548018000,-96.799971,-284.349112,-100.740786,-274.796798,-106.158246,-295.90179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
670020,2.0,0,1546808400,167.280029,110.140888,153.339214,57.533202,147.921754,36.42821,-2.00618,...,-977.373846,-613.770792,-14.996269,7.369552,-180.747724,-19.832889,0.305572,-12.175933,-0.45614,0.0


In [14]:
X = features_train.drop(['target'], axis=1)
y = (features_train['target'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=18)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_valid_scale = scaler.transform(X_valid)

In [69]:
def train_model(model, X_train, y_train, X_valid, y_valid, feature_names):
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    
    print("f1: {}".format(f1_score(y_valid, y_pred, average='macro')))
    
    print(pd.DataFrame(model.coef_[0], index=feature_names, 
                       columns=['weight']).sort_values(by='weight', ascending=False))
    
    report = classification_report(y_valid, y_pred, target_names=['No', 'Yes'])
    print(report)

In [70]:
logreg = LogisticRegression()
train_model(logreg, X_train_scale, y_train, X_valid_scale, y_valid, X_train.columns)

C:\Users\gdu19\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


f1: 0.4823434747566901
          weight
vas_id  0.847911
137     0.053017
192     0.046777
130     0.046163
50      0.038520
...          ...
15     -0.059017
135    -0.059166
213    -0.062974
24     -0.069094
13     -0.083204

[255 rows x 1 columns]
              precision    recall  f1-score   support

          No       0.93      1.00      0.96    230578
         Yes       0.12      0.00      0.00     17186

    accuracy                           0.93    247764
   macro avg       0.53      0.50      0.48    247764
weighted avg       0.87      0.93      0.90    247764



In [71]:
?classification_report

In [72]:
# Читаем тестовые данные
# data_test = pd.read_csv('data_test.csv')
# data_test.pop('Unnamed: 0')
# data_test.head()

In [73]:
?f1_score